In [1]:
import pandas as pd
from pathlib import Path

csv_dir = Path("../data/raw/MachineLearningCVE")

csv_files = list(csv_dir.glob("*.csv"))
len(csv_files), [f.name for f in csv_files]


(8,
 ['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
  'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
  'Friday-WorkingHours-Morning.pcap_ISCX.csv',
  'Monday-WorkingHours.pcap_ISCX.csv',
  'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
  'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
  'Tuesday-WorkingHours.pcap_ISCX.csv',
  'Wednesday-workingHours.pcap_ISCX.csv'])

In [2]:
dfs = {}
for f in csv_files:
    df = pd.read_csv(f)
    dfs[f.name] = df
    print(f.name, df.shape)

Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv (225745, 79)
Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv (286467, 79)
Friday-WorkingHours-Morning.pcap_ISCX.csv (191033, 79)
Monday-WorkingHours.pcap_ISCX.csv (529918, 79)
Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv (288602, 79)
Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv (170366, 79)
Tuesday-WorkingHours.pcap_ISCX.csv (445909, 79)
Wednesday-workingHours.pcap_ISCX.csv (692703, 79)


In [3]:
def normalize_columns(df):
    df.columns = df.columns.str.strip()
    return df

for k in dfs:
    dfs[k] = normalize_columns(dfs[k])


In [4]:
dfs[csv_files[0].name].columns.tolist()


['Destination Port',
 'Flow Duration',
 'Total Fwd Packets',
 'Total Backward Packets',
 'Total Length of Fwd Packets',
 'Total Length of Bwd Packets',
 'Fwd Packet Length Max',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Min',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Flow Bytes/s',
 'Flow Packets/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Flow IAT Min',
 'Fwd IAT Total',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Fwd IAT Min',
 'Bwd IAT Total',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'Fwd Header Length',
 'Bwd Header Length',
 'Fwd Packets/s',
 'Bwd Packets/s',
 'Min Packet Length',
 'Max Packet Length',
 'Packet Length Mean',
 'Packet Length Std',
 'Packet Length Variance',
 'FIN Flag Count',
 'SYN Flag Count',
 'RST Flag Count',
 'PSH Flag Count',
 'ACK Flag Count',
 'UR

In [13]:
def infer_day(filename):
    if "Monday" in filename: return "Monday"
    if "Tuesday" in filename: return "Tuesday"
    if "Wednesday" in filename: return "Wednesday"
    if "Thursday" in filename: return "Thursday"
    if "Friday" in filename: return "Friday"
    return "Unknown"

def infer_attack_group(filename):
    if "Benign" in filename or "Monday" in filename:
        return "benign"
    if "DoS" in filename or "DDoS" in filename:
        return "dos"
    if "PortScan" in filename:
        return "scan"
    if "WebAttack" in filename or "WebAttacks" in filename:
        return "web"
    if "Infilteration" in filename:
        return "infiltration"
    if "Bot" in filename:
        return "bot"
    return "mixed"


In [14]:
for name, df in dfs.items():
    df["source_file"] = name
    df["day"] = infer_day(name)
    df["attack_group"] = infer_attack_group(name)


In [15]:
dfs[name][["source_file", "day", "attack_group"]].head()


,source_file,day,attack_group
0,Wednesday-workingHours.pcap_ISCX.csv,Wednesday,mixed
1,Wednesday-workingHours.pcap_ISCX.csv,Wednesday,mixed
2,Wednesday-workingHours.pcap_ISCX.csv,Wednesday,mixed
3,Wednesday-workingHours.pcap_ISCX.csv,Wednesday,mixed
4,Wednesday-workingHours.pcap_ISCX.csv,Wednesday,mixed


In [16]:
merged_df = pd.concat(dfs.values(), ignore_index=True)
merged_df.shape


(2830743, 82)

In [17]:
merged_df["label_binary"] = (merged_df["Label"] != "BENIGN").astype(int)

In [18]:
merged_df["label_binary"].value_counts(normalize=True)


label_binary
0    0.803004
1    0.196996
Name: proportion, dtype: float64

In [19]:
out_path = Path("../data/processed/cicids_v1_raw.csv")
merged_df.to_csv(out_path, index=False)


In [20]:
merged_df["attack_group"].value_counts()


attack_group
mixed           1555390
benign           529918
infiltration     288602
scan             286467
web              170366
Name: count, dtype: int64

In [21]:
merged_df["Label"].value_counts().head(10)

Label
BENIGN              2273097
DoS Hulk             231073
PortScan             158930
DDoS                 128027
DoS GoldenEye         10293
FTP-Patator            7938
SSH-Patator            5897
DoS slowloris          5796
DoS Slowhttptest       5499
Bot                    1966
Name: count, dtype: int64